# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
from statsmodels.stats.power import TTestIndPower, TTestPower
import statsmodels as sm
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
clickedID = df[df['action']=='click']['id'].unique()
viewedID = df[df['action']=='view']['id'].unique()
print('Viewer count: {}'.format(len(viewedID)))
print('Clicker count: {}'.format(len(clickedID)))
print('Count of viewers who did not click: {}'.format(len(viewedID)-len(clickedID)))

exptID = set(df[df['group']=='experiment']['id'].unique())
ctrlID = set(df[df['group']=='control']['id'].unique())
print('\nIndividuals in expt group: {}'.format(len(exptID)))
print('Individuals in ctrl group: {}'.format(len(ctrlID)))
print('Individuals in both groups: {}'.format(len(exptID.intersection(ctrlID))))

print('Number of clicked who also viewed: {}'.format(len(set(clickedID).intersection(set(viewedID)))))

Viewer count: 6328
Clicker count: 1860
Count of viewers who did not click: 4468

Individuals in expt group: 2996
Individuals in ctrl group: 3332
Individuals in both groups: 0
Number of clicked who also viewed: 1860


In [3]:
"""
The majority of the website viewers didn't click on a link.

The number of individuals if approximately similar between experimental and control groups.
Futher, there is no overlap in individuals between these groups.

All individuals who clicked also viewed."""

"\nThe majority of the website viewers didn't click on a link.\n\nThe number of individuals if approximately similar between experimental and control groups.\nFuther, there is no overlap in individuals between these groups.\n\nAll individuals who clicked also viewed."

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [39]:
#Your code here
clickedID = set(clickedID)
viewedID = set(viewedID)
exptID = set(exptID)
ctrlID = set(ctrlID)

contingency_table = [[len(exptID.intersection(clickedID)), len(exptID.intersection(viewedID))],
                    [len(ctrlID.intersection(clickedID)), len(ctrlID.intersection(viewedID))]]
contingency_table
results = stats.chi2_contingency(contingency_table)
results
chi_stat=results[0]
p_value = results[1]
dof = results[2]
expected_freq = results[3]

print(results[1]/2) #take half the p_value for a one-sided test
print('We can reject the null hypothesis that the control and experimental homepages had the same outcome on viewers who clicked, since the p_value of {:.4f} is less than our alpha level of 0.05.'.format(p_value/2))


0.028268595543457887
We can reject the null hypothesis that the control and experimental homepages had the same outcome on viewers who clicked, since the p_value of 0.0283 is less than our alpha level of 0.05.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [23]:
#Your code here
exptID_click = pd.DataFrame([]) #initialize data frames
ctrlID_click = pd.DataFrame([])

for person in exptID: #loop over people in experimental group
    actions = list(df[df['id']==person]['action'])
    for action in actions: #for each action
        exptID_click.loc[person, actions] = 1 #store flag of 1
exptID_click.fillna(0, inplace=True) #fill NaNs with 0

for person in ctrlID: #loop over people in control group
    actions = list(df[df['id']==person]['action'])
    for action in actions: #for each action
        ctrlID_click.loc[person, actions] = 1 #store flag of 1
ctrlID_click.fillna(0, inplace=True) # fill NaNs with 0

print(exptID_click.describe()) #summarize views/clicks per group
print(ctrlID_click.describe())


ctrl_click_rate = ctrlID_click.click.mean()
expt_clicks_using_ctrl_rate = len(exptID_click)*ctrl_click_rate

print('The control group click rate was {} clicks per view.'.format(ctrl_click_rate))
print('The experimental group would have {} clicks if it had the same rate as the control group.'.format(expt_clicks_using_ctrl_rate))

         view        click
count  2996.0  2996.000000
mean      1.0     0.309746
std       0.0     0.462466
min       1.0     0.000000
25%       1.0     0.000000
50%       1.0     0.000000
75%       1.0     1.000000
max       1.0     1.000000
         view        click
count  3332.0  3332.000000
mean      1.0     0.279712
std       0.0     0.448925
min       1.0     0.000000
25%       1.0     0.000000
50%       1.0     0.000000
75%       1.0     1.000000
max       1.0     1.000000
The control group click rate was 0.2797118847539016 clicks per view.
The experimental group would have 838.0168067226891 clicks if it had the same rate as the control group.


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [32]:
#Your code here
expected_var = len(exptID_click)*ctrl_click_rate*(1 - ctrl_click_rate)
expected_std = np.sqrt(expected_var)
print('The expected standard deviation of clicks in the experimental group, using the control rate is {}'.format(expected_std))

print('Actual # of clicks in experimental group: {}'.format(exptID_click.click.sum()))
print('Expected # of clicks in experimental group (using control rate): {}'.format(expt_clicks_using_ctrl_rate))

z_value = (exptID_click.click.sum() - expt_clicks_using_ctrl_rate) / expected_std
                                             
print('(Actual - Expected) / std: {}'.format(z_value))


The expected standard deviation of clicks in the experimental group, using the control rate is 24.568547907005815
Actual # of clicks in experimental group: 928.0
Expected # of clicks in experimental group (using control rate): 838.0168067226891
(Actual - Expected) / std: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [36]:
#Your code here
print(1 - stats.norm.cdf(z_value))

0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Both approaches lead us to reject the null hypothesis of the rate of clicks being the same between experimental and control groups.**
The second analysis leads to lower estimated p_values however. Both analyses lead us to conclude that the experimental page leads a greater fraction of viewers to initiate clicks.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.